# 🔧 Configuração do Spark - SOLUÇÃO PARA CONNECTION REFUSED

**Execute este notebook primeiro antes de usar Spark em outros notebooks!**

Este notebook resolve os problemas de:
- ❌ `[Errno 111] Connection refused`
- ❌ `Py4JNetworkError: Answer from Java side is empty`

**Melhorias incluídas:**
- ✅ Verificação e liberação automática de portas
- ✅ Portas dinâmicas para evitar conflitos
- ✅ Configurações robustas de rede e timeout
- ✅ Detecção automática de problemas

In [83]:
# PASSO 1: Limpar ambiente e processos Spark COMPLETAMENTE
import os
import sys
import subprocess
import time
import gc

print("=" * 80)
print("🔧 CONFIGURAÇÃO SPARK - CORREÇÃO CONNECTION REFUSED")
print("=" * 80)

# Limpar TODAS as sessões Spark existentes de forma agressiva
try:
    from pyspark.sql import SparkSession
    from pyspark import SparkContext
    
    # Tentar parar todas as sessões ativas
    try:
        active_session = SparkSession.getActiveSession()
        if active_session:
            print("ℹ️  Parando Spark Session ativa...")
            try:
                active_session.sparkContext.stop()
            except:
                pass
            try:
                active_session.stop()
            except:
                pass
            time.sleep(2)
    except:
        pass
    
    # Tentar parar SparkContext global se existir
    try:
        sc = SparkContext._active_spark_context
        if sc:
            print("ℹ️  Parando SparkContext global...")
            sc.stop()
            time.sleep(2)
    except:
        pass
    
    # Limpar referências globais do PySpark
    try:
        SparkSession._instantiatedContext = None
        SparkContext._active_spark_context = None
    except:
        pass
    
    print("✅ Sessões Spark paradas")
except ImportError:
    print("ℹ️  PySpark não importado ainda")
except Exception as e:
    print(f"⚠️  Erro ao limpar sessões: {e}")

# Matar TODOS os processos Java relacionados ao Spark
print("\nℹ️  Matando processos Java do Spark...")
for pattern in ["org.apache.spark", "java.*spark", "SparkSubmit"]:
    try:
        result = subprocess.run(
            ["pgrep", "-f", pattern],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0:
            pids = result.stdout.strip().split('\n')
            for pid in pids:
                if pid and pid.isdigit():
                    try:
                        print(f"  ℹ️  Matando processo PID: {pid}")
                        subprocess.run(["kill", "-9", pid], check=False, timeout=2)
                    except:
                        pass
    except:
        pass

time.sleep(3)  # Aguardar processos terminarem completamente

# Limpar variáveis globais Python que podem manter referências
try:
    import __main__
    if hasattr(__main__, 'spark'):
        delattr(__main__, 'spark')
    if hasattr(__main__, 'sc'):
        delattr(__main__, 'sc')
except:
    pass

# Forçar garbage collection múltiplas vezes
for _ in range(3):
    gc.collect()
    time.sleep(0.5)

print("✅ Ambiente completamente limpo")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


🔧 CONFIGURAÇÃO SPARK - CORREÇÃO CONNECTION REFUSED
ℹ️  Parando SparkContext global...
✅ Sessões Spark paradas

ℹ️  Matando processos Java do Spark...


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


✅ Ambiente completamente limpo


In [84]:
# PASSO 2: Verificar e liberar portas em uso
import socket

def check_port(port):
    """Verifica se uma porta está em uso"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex(('127.0.0.1', port))
        sock.close()
        return result == 0
    except:
        return False

def kill_process_on_port(port):
    """Mata processo usando uma porta específica"""
    try:
        # Tentar lsof
        result = subprocess.run(
            ["lsof", "-ti", f":{port}"],
            capture_output=True,
            text=True,
            timeout=2
        )
        if result.returncode == 0 and result.stdout.strip():
            pids = result.stdout.strip().split('\n')
            for pid in pids:
                if pid and pid.isdigit():
                    try:
                        subprocess.run(["kill", "-9", pid], check=False, timeout=2)
                    except:
                        pass
            time.sleep(1)
            return True
    except:
        pass
    return False

# Verificar e liberar portas comuns do Spark
spark_ports = [4040, 4041, 4042, 8080, 8081, 7077, 7078]
portas_liberadas = []
for port in spark_ports:
    if check_port(port):
        print(f"⚠️  Porta {port} está em uso")
        if kill_process_on_port(port):
            if not check_port(port):
                print(f"✅ Porta {port} liberada")
                portas_liberadas.append(port)

if portas_liberadas:
    print(f"✅ {len(portas_liberadas)} portas liberadas")
else:
    print("✅ Nenhuma porta em conflito encontrada")

✅ Nenhuma porta em conflito encontrada


In [85]:
# PASSO 3: Configurar variáveis de ambiente
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'

# Verificar JAVA_HOME
java_home_candidates = [
    '/usr/lib/jvm/java-17-openjdk-amd64',
    '/usr/lib/jvm/java-11-openjdk-amd64',
    '/usr/lib/jvm/java-8-openjdk-amd64',
    os.environ.get('JAVA_HOME', ''),
]

java_home = None
for candidate in java_home_candidates:
    if candidate and os.path.exists(candidate):
        java_home = candidate
        break

if java_home:
    os.environ['JAVA_HOME'] = java_home
    print(f"✅ JAVA_HOME configurado: {java_home}")
else:
    print("⚠️  JAVA_HOME não encontrado, tentando detectar...")
    try:
        result = subprocess.run(["which", "java"], capture_output=True, text=True)
        if result.returncode == 0:
            java_path = result.stdout.strip()
            java_home = os.path.dirname(os.path.dirname(java_path))
            os.environ['JAVA_HOME'] = java_home
            print(f"✅ JAVA_HOME detectado: {java_home}")
    except:
        pass

# Configurações de rede
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['SPARK_MASTER'] = 'local[*]'
os.environ['SPARK_DRIVER_HOST'] = '127.0.0.1'
os.environ['SPARK_DRIVER_BIND_ADDRESS'] = '127.0.0.1'
os.environ['SPARK_LOCAL_DIRS'] = '/tmp/spark-local'

os.makedirs('/tmp/spark-local', exist_ok=True)
print("✅ Variáveis de ambiente configuradas")

✅ JAVA_HOME configurado: /usr/lib/jvm/java-17-openjdk-amd64
✅ Variáveis de ambiente configuradas


In [86]:
# PASSO 4: Importar PySpark
from pyspark.sql import SparkSession
import pyspark

print("✅ PySpark importado")
print(f"ℹ️  Versão Spark: {pyspark.__version__}")
print("ℹ️  Delta Lake será carregado via JARs")

✅ PySpark importado
ℹ️  Versão Spark: 3.5.0
ℹ️  Delta Lake será carregado via JARs


In [87]:
# PASSO 5: Configurações MinIO e versões
MINIO_ENDPOINT = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "1q2w3e4r"
BUCKET_NAME = "govbr"

# Determinar versão do Delta Lake compatível
spark_version = pyspark.__version__
if spark_version.startswith("4."):
    delta_package = "io.delta:delta-spark_2.13:4.0.0"
    hadoop_package = "org.apache.hadoop:hadoop-aws:3.3.4"
    aws_package = "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    print(f"✅ Usando Delta Lake 4.0.0 para Spark 4.x")
elif spark_version.startswith("3.5"):
    delta_package = "io.delta:delta-spark_2.12:3.0.0"
    hadoop_package = "org.apache.hadoop:hadoop-aws:3.3.4"
    aws_package = "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    print(f"✅ Usando Delta Lake 3.0.0 para Spark 3.5.x")
else:
    delta_package = "io.delta:delta-spark_2.12:2.4.0"
    hadoop_package = "org.apache.hadoop:hadoop-aws:3.3.4"
    aws_package = "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    print(f"✅ Usando Delta Lake 2.4.0 (padrão)")

print("✅ Configurações definidas")

✅ Usando Delta Lake 3.0.0 para Spark 3.5.x
✅ Configurações definidas


In [88]:
# PASSO 6: Verificar Java antes de criar Builder
print("ℹ️  Verificando Java...")
try:
    result = subprocess.run(["java", "-version"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("✅ Java está disponível")
        print(f"   {result.stderr.split(chr(10))[0] if result.stderr else 'Versão detectada'}")
    else:
        print("⚠️  Java pode não estar configurado corretamente")
except Exception as e:
    print(f"⚠️  Erro ao verificar Java: {e}")

# Verificar JAVA_HOME
java_home = os.environ.get('JAVA_HOME')
if java_home:
    print(f"✅ JAVA_HOME: {java_home}")
    if os.path.exists(java_home):
        print("   ✅ JAVA_HOME válido")
    else:
        print("   ⚠️  JAVA_HOME aponta para diretório inexistente")
else:
    print("⚠️  JAVA_HOME não configurado")

# PASSO 6b: Criar Spark Builder com configurações robustas
# ⭐ IMPORTANTE: Portas dinâmicas (0) evitam "Connection refused"
print("\nℹ️  Configurando Spark Builder...")
builder = SparkSession.builder \
    .appName("GovBR Data Lake") \
    .master("local[*]") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.port", "0") \
    .config("spark.blockManager.port", "0") \
    .config("spark.broadcast.port", "0") \
    .config("spark.fileserver.port", "0") \
    .config("spark.replClassServer.port", "0") \
    .config("spark.ui.port", "0") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.network.timeout", "1200s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.jars.packages", f"{delta_package},{hadoop_package},{aws_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", f"https://{MINIO_ENDPOINT}") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.connection.maximum", "15") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.attempts.maximum", "10") \
    .config("spark.driver.extraJavaOptions", 
            "-Dio.netty.tryReflectionSetAccessible=true " +
            "-XX:+UseG1GC " +
            "-XX:MaxGCPauseMillis=200 " +
            "-Djava.net.preferIPv4Stack=true " +
            "-Djava.awt.headless=true") \
    .config("spark.executor.extraJavaOptions", 
            "-Dio.netty.tryReflectionSetAccessible=true " +
            "-XX:+UseG1GC " +
            "-Djava.net.preferIPv4Stack=true " +
            "-Djava.awt.headless=true") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")

print("✅ Builder configurado com:")
print("   - Portas dinâmicas (evita Connection refused)")
print("   - Timeouts aumentados")
print("   - Garbage Collector otimizado")
print("   - Suporte a S3A (MinIO)")

ℹ️  Verificando Java...
✅ Java está disponível
   openjdk version "17.0.8.1" 2023-08-24
✅ JAVA_HOME: /usr/lib/jvm/java-17-openjdk-amd64
   ✅ JAVA_HOME válido

ℹ️  Configurando Spark Builder...
✅ Builder configurado com:
   - Portas dinâmicas (evita Connection refused)
   - Timeouts aumentados
   - Garbage Collector otimizado
   - Suporte a S3A (MinIO)


In [89]:
# PASSO 7: Verificar e inicializar JVM antes de criar Spark Session
print("ℹ️  Verificando ambiente Java antes de criar Spark Session...")

# Verificar se Java está realmente disponível
java_ok = False
try:
    result = subprocess.run(["java", "-version"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0 or result.stderr:
        java_ok = True
        print("✅ Java está disponível")
    else:
        print("⚠️  Java pode não estar funcionando corretamente")
except Exception as e:
    print(f"❌ Erro ao verificar Java: {e}")
    print("\n💡 CRÍTICO: Java não está disponível!")
    print("   Execute: sudo apt-get update && sudo apt-get install -y openjdk-17-jdk")
    raise RuntimeError("Java não está disponível. Instale Java antes de continuar.")

# Verificar JAVA_HOME
java_home = os.environ.get('JAVA_HOME')
if not java_home or not os.path.exists(java_home):
    print("⚠️  JAVA_HOME não configurado ou inválido")
    # Tentar detectar automaticamente
    try:
        result = subprocess.run(["which", "java"], capture_output=True, text=True)
        if result.returncode == 0:
            java_path = result.stdout.strip()
            java_home = os.path.dirname(os.path.dirname(java_path))
            os.environ['JAVA_HOME'] = java_home
            print(f"✅ JAVA_HOME detectado: {java_home}")
    except:
        pass

# Limpar completamente qualquer referência ao SparkContext antes de criar
print("\nℹ️  Limpeza final antes de criar sessão...")
try:
    from pyspark import SparkContext
    # Forçar limpeza do SparkContext
    if hasattr(SparkContext, '_active_spark_context'):
        SparkContext._active_spark_context = None
    if hasattr(SparkContext, '_jvm'):
        SparkContext._jvm = None
    if hasattr(SparkContext, '_gateway'):
        SparkContext._gateway = None
except:
    pass

# Limpar referências de SparkSession
try:
    from pyspark.sql import SparkSession
    existing = SparkSession.getActiveSession()
    if existing:
        print("⚠️  ATENÇÃO: Ainda há uma sessão ativa! Parando...")
        try:
            existing.stop()
            time.sleep(3)
        except:
            pass
    # Limpar referências globais
    SparkSession._instantiatedContext = None
    SparkSession._instantiatedSession = None
except:
    pass

# Forçar garbage collection antes de criar
for _ in range(3):
    gc.collect()
    time.sleep(0.5)

print("✅ Ambiente limpo. Criando Spark Session...")
print("ℹ️  Usando portas dinâmicas para evitar Connection refused")

try:
    # ⭐ CRÍTICO: Usar builder diretamente, mas garantir que JVM seja inicializado
    print("ℹ️  Criando Spark Session (JVM será inicializado automaticamente)...")
    
    # O builder.getOrCreate() vai criar o SparkConf internamente
    # e isso vai inicializar o JVM através do Py4J
    # Mas precisamos garantir que não há conflitos
    
    # Criar sessão diretamente usando o builder
    # O PySpark vai inicializar o JVM automaticamente quando necessário
    spark = builder.getOrCreate()
    
    # Aguardar Spark inicializar completamente
    print("ℹ️  Aguardando Spark inicializar completamente...")
    time.sleep(5)
    
    # Verificar se Spark está respondendo
    print("ℹ️  Verificando conectividade...")
    version = spark.version
    app_name = spark.sparkContext.appName
    master = spark.sparkContext.master
    
    print(f"\n✅ Spark Session criada com sucesso!")
    print(f"   Versão: {version}")
    print(f"   App Name: {app_name}")
    print(f"   Master: {master}")
    
except ConnectionRefusedError as e:
    print(f"\n❌ Connection Refused: O servidor Java do Spark não está respondendo")
    print(f"   Erro: {e}")
    print("\n💡 DIAGNÓSTICO:")
    print("1. Verifique processos Java:")
    print("   !ps aux | grep java")
    print("2. Verifique portas:")
    print("   !netstat -tuln | grep -E '4040|8080|7077'")
    print("3. Verifique JAVA_HOME:")
    print(f"   !echo $JAVA_HOME")
    print("4. Reinicie o container:")
    print("   docker restart govbr-jupyter-delta")
    print("5. Execute o script completo:")
    print("   exec(open('/home/jovyan/work/fix_spark_py4j.py').read())")
    raise
    
except Exception as e:
    print(f"\n❌ Erro ao criar Spark Session: {e}")
    import traceback
    traceback.print_exc()
    
    print("\n" + "=" * 80)
    print("TENTANDO MODO FALLBACK (sem Delta, configuração mínima)...")
    print("=" * 80)
    
    # Fallback: criar Spark Session com configuração mínima
    try:
        conf_fallback = SparkConf()
        conf_fallback.set("spark.app.name", "GovBR Data Lake (Fallback)")
        conf_fallback.set("spark.master", "local[*]")
        conf_fallback.set("spark.driver.host", "127.0.0.1")
        conf_fallback.set("spark.driver.bindAddress", "127.0.0.1")
        conf_fallback.set("spark.driver.port", "0")
        conf_fallback.set("spark.blockManager.port", "0")
        conf_fallback.set("spark.ui.port", "0")
        conf_fallback.set("spark.driver.memory", "2g")
        conf_fallback.set("spark.executor.memory", "2g")
        conf_fallback.set("spark.network.timeout", "1200s")
        conf_fallback.set("spark.driver.extraJavaOptions", 
                "-Dio.netty.tryReflectionSetAccessible=true " +
                "-Djava.net.preferIPv4Stack=true " +
                "-Djava.awt.headless=true")
        
        print("ℹ️  Criando SparkConf fallback...")
        time.sleep(2)
        
        print("ℹ️  Criando Spark Session fallback...")
        spark = SparkSession.builder.config(conf=conf_fallback).getOrCreate()
        time.sleep(5)
        
        version = spark.version
        print("✅ Spark Session criada em modo fallback (sem Delta)")
        print(f"   Versão: {version}")
        print("⚠️  ATENÇÃO: Delta Lake não está disponível. Use apenas Parquet.")
    except Exception as e3:
        print(f"❌ Erro também no modo fallback: {e3}")
        print("\n💡 SOLUÇÕES CRÍTICAS:")
        print("1. Reinicie o container: docker restart govbr-jupyter-delta")
        print("2. Verifique se Java está instalado: java -version")
        print("3. Verifique JAVA_HOME: echo $JAVA_HOME")
        print("4. Execute o script completo: exec(open('/home/jovyan/work/fix_spark_py4j.py').read())")
        print("5. Verifique logs: docker logs govbr-jupyter-delta")
        raise

ℹ️  Verificando ambiente Java antes de criar Spark Session...
✅ Java está disponível

ℹ️  Limpeza final antes de criar sessão...
✅ Ambiente limpo. Criando Spark Session...
ℹ️  Usando portas dinâmicas para evitar Connection refused
ℹ️  Criando Spark Session (JVM será inicializado automaticamente)...
ℹ️  Aguardando Spark inicializar completamente...
ℹ️  Verificando conectividade...

✅ Spark Session criada com sucesso!
   Versão: 3.5.0
   App Name: GovBR Data Lake
   Master: local[*]


In [90]:
# PASSO 8: Teste de conectividade Py4J
print("\n" + "=" * 80)
print("TESTE DE CONECTIVIDADE")
print("=" * 80)

try:
    test_df = spark.range(5)
    count = test_df.count()
    if count == 5:
        print(f"✅ Teste Py4J bem-sucedido! ({count} registros)")
        test_df.show()
    else:
        print(f"⚠️  Teste retornou resultado inesperado: {count}")
except Exception as e:
    print(f"❌ Erro no teste Py4J: {e}")
    print("💡 Execute o notebook FIX_PY4J_ERROR.ipynb para correção completa")
    raise


TESTE DE CONECTIVIDADE
✅ Teste Py4J bem-sucedido! (5 registros)
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [91]:
# Tornar spark disponível globalmente
import __main__
__main__.spark = spark
globals()['spark'] = spark

print("\n" + "=" * 80)
print("✅ SPARK SESSION PRONTA PARA USO!")
print("=" * 80)
print("\nA variável 'spark' está disponível para uso nos notebooks.")
print("Use spark para criar DataFrames e executar consultas SQL.")


✅ SPARK SESSION PRONTA PARA USO!

A variável 'spark' está disponível para uso nos notebooks.
Use spark para criar DataFrames e executar consultas SQL.


## ✅ Pronto!

Agora você pode usar o Spark normalmente em outros notebooks. A variável `spark` está disponível.

### Exemplo de uso:

```python
# Ler dados do MinIO
df = spark.read.parquet("s3a://govbr/bronze/ibge/municipios/")
df.show()

# Executar consultas SQL
df.createOrReplaceTempView("municipios")
result = spark.sql("SELECT * FROM municipios LIMIT 10")
result.show()
```

In [92]:
# Ler dados do MinIO
df = spark.read.parquet("s3a://govbr/bronze/ibge/municipios/")
df.show()

# Executar consultas SQL
df.createOrReplaceTempView("municipios")
result = spark.sql("SELECT * FROM municipios LIMIT 10")
result.show()

+-----------+--------------------+--------+--------+---------+-----------+---------------+-----------------+--------------+-----------------+--------+
|codigo_ibge|           municipio|uf_sigla| uf_nome|regiao_id|regiao_nome|microrregiao_id|microrregiao_nome|mesorregiao_id| mesorregiao_nome|      dt|
+-----------+--------------------+--------+--------+---------+-----------+---------------+-----------------+--------------+-----------------+--------+
|    1100015|Alta Floresta D'O...|      RO|Rondônia|      1.0|      Norte|        11006.0|           Cacoal|        1102.0|Leste Rondoniense|20251114|
|    1100023|           Ariquemes|      RO|Rondônia|      1.0|      Norte|        11003.0|        Ariquemes|        1102.0|Leste Rondoniense|20251114|
|    1100031|              Cabixi|      RO|Rondônia|      1.0|      Norte|        11008.0|Colorado do Oeste|        1102.0|Leste Rondoniense|20251114|
|    1100049|              Cacoal|      RO|Rondônia|      1.0|      Norte|        11006.0|    